In [1]:
# !pip install -r https://raw.githubusercontent.com/anujavenkatachalam04/chvi_vbd_rj/main/requirements.txt

In [2]:
import os
import pandas as pd
import geopandas as gpd
import requests
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive
drive.mount('/content/drive')
import geopandas as gpd
from shapely.geometry import Point
import uuid


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
os.chdir("/content/drive/MyDrive/CHVI")


In [4]:
blocks_shp=gpd.read_file("5_Shapefiles/SUBDISTRICT_11/Rajasthan_Blocks.geojson")

In [5]:
grid=gpd.read_file("5_Shapefiles/Rajasthan_10kmx10km_grid.geojson")

In [6]:
import os
import pandas as pd
import requests_cache
from retry_requests import retry
from dotenv import load_dotenv
import openmeteo_requests

# Load API key from .env  only works for gh
# load_dotenv()
# OPENMETEO_API_KEY = os.getenv("OPENMETEO_API_KEY")

In [11]:
grid

,NAME,DISTRICT,C_CODE11,geometry
0,Jaisalmer,Jaisalmer,0811400574000000,POINT (-38900.543 2970335.012)
1,Jaisalmer,Jaisalmer,0811400574000000,POINT (-38900.543 2980335.012)
2,Jaisalmer,Jaisalmer,0811400574000000,POINT (-38900.543 2990335.012)
3,Jaisalmer,Jaisalmer,0811400574000000,POINT (-38900.543 3000335.012)
4,Jaisalmer,Jaisalmer,0811400574000000,POINT (-38900.543 3010335.012)
...,...,...,...,...
3415,Rajakhera,Dhaulpur,0810600519000000,POINT (801099.457 2960335.012)
3416,Rajakhera,Dhaulpur,0810600519000000,POINT (801099.457 2970335.012)
3417,Rajakhera,Dhaulpur,0810600519000000,POINT (811099.457 2970335.012)
3418,Rajakhera,Dhaulpur,0810600519000000,POINT (811099.457 2980335.012)


# Extract ERA5 data at 10km x 10km resolution

In [8]:
def fetch_era5_for_blocks(blocks_gdf, start_date, end_date, api_key, daily_vars=None):
    """
    Fetch daily ERA5 data for multiple blocks using their centroids.

    Parameters:
        blocks_gdf (GeoDataFrame): must have columns 'NAME', 'lat', 'lon'
        start_date (str): "YYYY-MM-DD"
        end_date (str): "YYYY-MM-DD"
        daily_vars (list): list of daily variable names (default = VBD relevant)

    Returns:
        pd.DataFrame: combined daily ERA5 data with block info
    """

    if daily_vars is None:
        daily_vars = [
            "temperature_2m_mean","temperature_2m_max","temperature_2m_min",
            "rain_sum","precipitation_sum","relative_humidity_2m_mean",
            "apparent_temperature_mean","precipitation_hours",
            "apparent_temperature_max","apparent_temperature_min",
            "et0_fao_evapotranspiration",
            "wind_speed_10m_max","wind_gusts_10m_max","wind_direction_10m_dominant",
            "shortwave_radiation_sum","sunshine_duration"
        ]

    # Setup Open-Meteo client
    cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
    retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
    client = openmeteo_requests.Client(session=retry_session)

    url = "https://customer-archive-api.open-meteo.com/v1/archive"

    all_data = []

    for idx, row in blocks_gdf.iterrows():
        lat, lon = row["lat"], row["lon"]
        params = {
            "latitude": lat,
            "longitude": lon,
            "start_date": start_date,
            "end_date": end_date,
            "daily": daily_vars,
            "models": "era5",
            "apikey": api_key
        }
        responses = client.weather_api(url, params=params)
        if not responses:
            print(f"No data for block {row['NAME']}")
            continue
        response = responses[0]
        daily = response.Daily()

        daily_index = pd.date_range(
            start=pd.to_datetime(daily.Time(), unit="s", utc=True),
            end=pd.to_datetime(daily.TimeEnd(), unit="s", utc=True),
            freq=pd.Timedelta(seconds=daily.Interval()),
            inclusive="left"
        )

        daily_data = {"date": daily_index, "NAME": row["NAME"], "DISTRICT": row["DISTRICT"], "C_CODE11": row["C_CODE11"]}
        for i, var in enumerate(daily_vars):
            daily_data[var] = daily.Variables(i).ValuesAsNumpy()

        all_data.append(pd.DataFrame(daily_data))

    combined_df = pd.concat(all_data, ignore_index=True)
    return combined_df


In [10]:
start_date = "2024-01-01"
end_date = "2025-10-01"
api_key="FrBHTVq25VDS9Lfu"
era5_df = fetch_era5_for_blocks(blocks_shp, start_date, end_date, api_key)
print(era5_df.head())

KeyError: 'lat'

In [ ]:
weather_df.to_csv("climate_data/openmeteo_era5_grid_data_10kmx10km_2024-01-01_to_2025-09-30.csv", index=False)

In [ ]:
def get_season(month):
    if month in [3, 4, 5]:
        return 'Summer (Mar-May)'
    elif month in [6, 7, 8]:
        return 'Monsoon (Jun-Aug)'
    elif month in [9, 10, 11]:
        return 'Post-Monsoon (Sep-Nov)'
    else:
        return 'Winter (Dec-Feb)'

In [ ]:
all_weather.drop(columns=["month_num"], inplace=True)

In [ ]:
all_weather['season'] = all_weather["time"].dt.month.apply(get_season)

In [ ]:
all_weather["diurnal_temperature_range"]=all_weather["temperature_2m_max"]-all_weather["temperature_2m_min"]

In [ ]:
all_weather.to_csv("climate_data/openmeteo_era5_grid_data_10kmx10km_2019-01-01_to_2025-06-29.csv", index=False)

# Climate - Aggregate API data (at grid-level) to block & day

In [ ]:
all_weather=pd.read_csv("climate_data/openmeteo_era5_grid_data_10kmx10km_2019-01-01_to_2025-06-29.csv")

In [ ]:
all_weather["time"]=pd.to_datetime(all_weather["time"])

In [ ]:
group_cols = ["dtname", "sdtname", "time"]

# Compute mean values per block and date
block_daily_climate = all_weather.groupby(group_cols).agg(
    {"temperature_2m_mean":"mean",
     "temperature_2m_max":"mean",
     "temperature_2m_min":"mean",
     "relative_humidity_2m_mean":"mean",
     "rain_sum":"sum",
     "diurnal_temperature_range":"mean"}
).reset_index()

In [ ]:
block_daily_climate.columns

Index(['dtname', 'sdtname', 'time', 'temperature_2m_mean',
       'temperature_2m_max', 'temperature_2m_min', 'relative_humidity_2m_mean',
       'rain_sum', 'diurnal_temperature_range'],
      dtype='object')

In [ ]:
block_daily_climate["time"].min(), block_daily_climate["time"].max()

(Timestamp('2019-01-01 00:00:00'), Timestamp('2025-06-29 00:00:00'))

In [ ]:
block_daily_climate["season"]=block_daily_climate["time"].dt.month.apply(get_season)

In [ ]:
block_daily_climate["season"].value_counts(normalize=True)

,proportion
season,
Summer,0.271501
Winter,0.253373
Monsoon,0.244941
Post-Monsoon,0.230185


In [ ]:
block_daily_climate.to_csv("dengue_malaria/clean/daily_climate_block.csv", index=False)

# Climate - Aggregate API data (at grid-level) to district & day

In [ ]:
all_weather=pd.read_csv("climate_data/openmeteo_era5_grid_data_10kmx10km_2019-01-01_to_2025-06-29.csv")

In [ ]:
all_weather["time"]=pd.to_datetime(all_weather["time"])

In [ ]:
agg_district = (
    all_weather.groupby(['dtname', 'time'])
    .agg({
        'temperature_2m_mean': 'mean',
        'relative_humidity_2m_mean': 'mean',
        'temperature_2m_max': 'mean',
        'temperature_2m_min': 'mean',
        'rain_sum': 'sum',
        'diurnal_temperature_range': 'mean'
    })
    .reset_index()
)


In [ ]:
agg_district["time"].min(), agg_district["time"].max()

(Timestamp('2019-01-01 00:00:00'), Timestamp('2025-06-29 00:00:00'))

In [ ]:
agg_district["season"]=agg_district["time"].dt.month.apply(get_season)

In [ ]:
agg_district["season"].value_counts(normalize=True)

,proportion
season,
Summer,0.271501
Winter,0.253373
Monsoon,0.244941
Post-Monsoon,0.230185


In [ ]:
agg_district.columns

Index(['dtname', 'time', 'temperature_2m_mean', 'relative_humidity_2m_mean',
       'temperature_2m_max', 'temperature_2m_min', 'rain_sum',
       'diurnal_temperature_range', 'season'],
      dtype='object')

In [ ]:
agg_district.to_csv("dengue_malaria/clean/daily_climate_district.csv", index=False)

In [ ]:
# The End!